In [1]:
!pip install numpy matplotlib tensorflow

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

c:\Users\mural\OneDrive\Desktop\classification_proj\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [ ]:
pip install kaggle


In [ ]:

!kaggle datasets download -d anandkumarsahu09/cattle-breeds-dataset

In [7]:
!tar -xf cattle-breeds-dataset.zip


In [3]:
datagen = ImageDataGenerator(
    rescale=1./255,          # normalize pixel values
    rotation_range=20,       # rotate images randomly
    width_shift_range=0.1,   # shift horizontally
    height_shift_range=0.1,  # shift vertically
    zoom_range=0.2,          # random zoomS
    horizontal_flip=True,    # flip horizontally
    validation_split=0.2     # 20% data for validation
)

# Training data
train_data = datagen.flow_from_directory(
    'cattle Breeds/',               # path to dataset folder
    target_size=(224,224),   # resize images
    batch_size=32,
    class_mode='categorical', # multi-class classification
    subset='training'
)

# Validation data
val_data = datagen.flow_from_directory(
    'cattle Breeds/',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 969 images belonging to 5 classes.
Found 239 images belonging to 5 classes.


In [4]:
print("Training samples:", train_data.samples)
print("Validation samples:", val_data.samples)
num_classes = train_data.num_classes
print("Number of breeds:", num_classes)

Training samples: 969
Validation samples: 239
Number of breeds: 5


In [5]:


model = Sequential([

    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


c:\Users\mural\OneDrive\Desktop\classification_proj\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,605 (42.61 MB)

 Trainable params: 11,169,605 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
!pip install scipy


In [7]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    batch_size=32,
    callbacks=[early_stop]
)

Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 513ms/step - accuracy: 0.2074 - loss: 1.7736 - val_accuracy: 0.2134 - val_loss: 1.5586
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 14s 443ms/step - accuracy: 0.2972 - loss: 1.4990 - val_accuracy: 0.3264 - val_loss: 1.4638
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 509ms/step - accuracy: 0.3571 - loss: 1.4531 - val_accuracy: 0.4059 - val_loss: 1.4575
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 533ms/step - accuracy: 0.3426 - loss: 1.4148 - val_accuracy: 0.3891 - val_loss: 1.3952
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 507ms/step - accuracy: 0.4025 - loss: 1.3548 - val_accuracy: 0.3640 - val_loss: 1.3796
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 516ms/step - accuracy: 0.3973 - loss: 1.3312 - val_accuracy: 0.4059 - val_loss: 1.3557
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 488ms/step - accuracy: 0.4293 - loss: 1.3016 - val_accuracy: 0.4603 - val_loss: 1.3212
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 481ms/step - accuracy: 0.4572 - loss: 1.2750 - val_accu

In [8]:
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy: {val_acc*100:.2f}%")

model.save("cow_breed_cnn.h5")
print("Model saved as cow_breed_cnn.keras")

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 214ms/step - accuracy: 0.4937 - loss: 1.2477


Validation Accuracy: 49.37%
Model saved as cow_breed_cnn.keras


In [14]:
y_true = val_data.classes
y_true


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype=int32)

In [11]:
y_pred_probs = model.predict(val_data)
y_pred_probs


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 210ms/step


array([[0.2734715 , 0.27904135, 0.02037651, 0.29676902, 0.13034157],
       [0.2656938 , 0.28282285, 0.08839063, 0.22003932, 0.14305341],
       [0.31985006, 0.19583678, 0.00344693, 0.1847829 , 0.2960833 ],
       ...,
       [0.39406273, 0.332488  , 0.05398371, 0.13412659, 0.08533893],
       [0.08258816, 0.02567778, 0.87768936, 0.01005084, 0.00399392],
       [0.24210572, 0.2117817 , 0.0057464 , 0.20030466, 0.3400615 ]],
      shape=(239, 5), dtype=float32)

In [13]:
import numpy as np

y_pred = np.argmax(y_pred_probs, axis=1)
y_pred


array([3, 1, 0, 4, 4, 1, 2, 1, 4, 2, 4, 2, 4, 2, 2, 0, 4, 0, 4, 0, 0, 4,
       0, 2, 4, 4, 1, 2, 2, 0, 0, 1, 4, 4, 0, 2, 2, 2, 4, 2, 4, 2, 0, 1,
       1, 4, 2, 2, 4, 2, 4, 0, 1, 1, 4, 0, 3, 2, 1, 4, 1, 2, 2, 1, 1, 1,
       4, 3, 3, 2, 1, 0, 4, 1, 2, 1, 3, 4, 2, 1, 4, 2, 1, 2, 2, 2, 0, 4,
       1, 4, 3, 4, 2, 1, 1, 2, 0, 3, 0, 1, 4, 1, 1, 0, 0, 1, 1, 1, 0, 4,
       3, 2, 2, 2, 1, 4, 0, 4, 2, 0, 1, 0, 3, 1, 4, 4, 1, 2, 2, 4, 3, 0,
       2, 4, 4, 1, 4, 1, 2, 2, 4, 3, 4, 0, 2, 1, 4, 4, 2, 1, 4, 0, 2, 4,
       0, 2, 2, 1, 2, 4, 2, 1, 2, 0, 4, 2, 1, 2, 2, 1, 1, 4, 3, 0, 1, 2,
       1, 2, 1, 2, 4, 2, 2, 1, 1, 1, 1, 1, 2, 0, 4, 2, 4, 1, 3, 4, 0, 4,
       0, 2, 4, 1, 4, 1, 4, 4, 4, 4, 4, 1, 4, 0, 2, 2, 1, 0, 4, 2, 2, 4,
       0, 2, 2, 2, 4, 1, 0, 2, 1, 2, 1, 1, 2, 2, 1, 0, 0, 2, 4])

In [16]:
!pip install scikit-learn
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)


  Using cached scikit_learn-1.8.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.8.0-cp312-cp312-win_amd64.whl (8.0 MB)
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- -

In [17]:
print(val_data.class_indices)


{'Ayrshire cattle': 0, 'Brown Swiss cattle': 1, 'Holstein Friesian cattle': 2, 'Jersey cattle': 3, 'Red Dane cattle': 4}


In [18]:
from sklearn.metrics import classification_report

print(classification_report(
    y_true,
    y_pred,
    target_names=list(val_data.class_indices.keys())
))


                          precision    recall  f1-score   support

         Ayrshire cattle       0.30      0.21      0.25        52
      Brown Swiss cattle       0.25      0.32      0.28        47
Holstein Friesian cattle       0.16      0.22      0.19        50
           Jersey cattle       0.15      0.04      0.06        50
         Red Dane cattle       0.19      0.30      0.24        40

                accuracy                           0.21       239
               macro avg       0.21      0.22      0.20       239
            weighted avg       0.21      0.21      0.20       239

